In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from utility import calculate_input_gradients, perturb_inputs, preprocess_images, \
                    postprocess_features, save_data_hdf5,get_dataset_hdf5,\
                    build_one_class_svm, combine_inliners_outliers, apply_temp_scale_to_model,\
                    apply_log_temp_scale_to_model, perturb_inputs_odin, extract_layer_features
 
from utility_db_outliers import load_dataset
from models_lib import load_custom_model_for_ds
import h5py
# from metrics2 import *
from metrics import *
from general_setting import *
import time
import tensorflow as tf
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Activation
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
#------------
from utility_methods import *
# from utility_methods2 import *
from sklearn.metrics import roc_auc_score

from tensorflow.keras.utils import to_categorical
import gzip
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import mnist
import scipy.io as sio
import tensorflow.keras.backend as K
import math

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import norm
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.layers import Conv2D, Flatten, Lambda
from tensorflow.keras.layers import Reshape, Conv2DTranspose, MaxPooling2D, UpSampling2D, AveragePooling2D
from tensorflow.keras.models import Model

In [ ]:
id_name=ID_DS_LIST[0] # selects the ID dataset.
id_model=ID_MODEL_LIST[0]  # select the deep model used for training ID dataset.
print(id_name,id_model)

In [ ]:
(org_traing_data, org_training_labels),(id_eva_data,org_testing_labels)  = load_dataset(id_name)

org_traing_data_processed = preprocess_images(id_name, org_traing_data, id_model, verbose=True)
id_eva_data_processed = preprocess_images(id_name, id_eva_data, id_model, verbose=True)

In [ ]:
x_train = org_traing_data_processed
y_train = org_training_labels
x_test = id_eva_data_processed
y_test = org_testing_labels

In [ ]:
# 网络参数
input_shape = (28, 28, 1)
batch_size = 256
kernel_size = 3
# structure = [16, "average", 32,"average"]
structure = [16, 32]
latent_dim = 784 # 隐变量取2维只是为了方便后面画图
epochs = 50
x_in = Input(shape=input_shape)
x = x_in

In [ ]:
for filters in structure:  
    if isinstance(filters, int):
        x = Conv2D(filters=filters,
                kernel_size=kernel_size,
                activation='relu',
                strides=2,
                padding='same')(x)
    # elif filters == "max":
    #     x = MaxPooling2D((2, 2), padding="same")(x)
    # elif filters == "average":
        # x = AveragePooling2D((2, 2), padding="same")(x)
#     print(x.shape)
x = Flatten()(x)
# print(x.shape)
x = Dense(latent_dim)(x)
# print(x.shape)

In [ ]:
x = Reshape((7,7,16))(x)
for filters in structure:  
    if isinstance(filters, int):
        x = Conv2DTranspose(filters=filters,
                    kernel_size=kernel_size,
                    activation='relu',
                    strides=2,
                    padding='same')(x)
        
    # elif filters == "max" or filters == "average":
    #     x = UpSampling2D((2, 2))(x)
    print(x.shape)

decoded = Conv2DTranspose(filters=1,
                        kernel_size=kernel_size,
                        activation='sigmoid',
                        padding='same')(x)
print(decoded.shape)

In [ ]:
ae = Model(x_in, decoded)

In [ ]:
# 搭建为一个独立的模型
ae.compile(loss='mean_squared_error',optimizer='rmsprop')

In [ ]:
ae.fit(x_train,x_train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size)
# ae.fit(x_train,x_train,
#         shuffle=True,
#         epochs=epochs,
#         batch_size=batch_size,
#         validation_data=(x_test, None))

In [ ]:
ae.save('./AE_models/%s_%s_1.h5'%(id_name,id_model))
ae.save_weights('./AE_models/%s_%s_weights_1.h5'%(id_name,id_model))

In [ ]:
vae = ae.load_weights('./AE_models/%s_%s_weights.h5'%(id_name,id_model))

In [ ]:
def mark(x_1, x_2):
    diff = np.abs(x_1 - x_2)
    marks = np.mean(np.power(diff, 2), axis=(1,2,3))
    return marks

In [ ]:
x_test = np.load('./advs_new/%s_%s_normal.npy'%(id_name, id_model),allow_pickle=True)
print(x_test.shape)

In [ ]:
sc_id = []
for i in range(len(x_test)):
    xx = ae.predict(x_test[i].reshape(1,28,28,1))
    # print(xx.shape)   
 
    score = mark(x_test[i],xx)
    # print(score)
    # break
    sc_id.append(-score)

In [ ]:
def combine_inliners_outliers(inliers, outliers, i_label=1, o_label=0, verbose=False):

    temp_outliers = outliers
    temp_inliers  = inliers
    if i_label==1:
        i_labels = np.ones(temp_inliers.shape[0])
    else:
        i_labels = np.zeros(temp_inliers.shape[0])
        
    if o_label==0:
        o_labels = np.zeros(temp_outliers.shape[0])
    else:
        o_labels = np.ones(temp_outliers.shape[0])       
              
    mixed_labels =  np.append(i_labels, o_labels)
    mixed_data = np.vstack((temp_inliers, temp_outliers))

    return mixed_data, mixed_labels

In [ ]:
test = np.asarray(sc_id)
ADV_DS_LIST = ("fgsm", "bim-a", "bim-b", "jsma", "cw-l2")
for i, ds_name in enumerate(ADV_DS_LIST):
    adv = id_name+"_"+id_model+"_"+ ds_name
    das = np.load('./advs_new/'+adv+'.npy',allow_pickle=True)
    sc_ood = []
    for i in range(len(das)):
        xx = ae.predict(das[i].reshape(1,28,28,1))
        score = mark(das[i],xx)
        sc_ood.append(-score)
    
    ood = np.asarray(sc_ood)
    scores, mixed_labels = combine_inliners_outliers(test, ood)
 
    fpr, tpr = nums(scores, mixed_labels)
    roc_auc = auc(fpr, tpr)
    lens = id_eva_data_processed.shape[0]
    FP,TN,TP,FN = ErrorRateAt95Recall1(lens, scores, mixed_labels)
    ROC = roc_auc_score(mixed_labels, scores, average='micro', sample_weight=None)
    #     print(get_summary_statistics(scores, mixed_labels))
    print('The results of %s:\n' %(ds_name))
    #     print('fpr_at_95_tpr:', float(FP) / float(FP + TN+ 1e-7) *100)
    print('tnr_at_95_tpr:', float(TN) / float(FP + TN+ 1e-7) *100)
    print('detection_acc:',(float(TP) / float(TP + FN + 1e-7)+ float(TN) / float(FP + TN + 1e-7))/2*100)
    #     print('detection_errror:',(1.0- float(TP) / float(TP + FN+ 1e-7)+ float(FP) / float(FP + TN+ 1e-7))/2*100)
    print("AUROC: ",ROC*100)
    print("\n")

In [ ]:
#layer_inx specifies the index of the OODL found by "find_oodl" jupyter file.
REM_TOP_LAYER = -2
NUM_CLASS = 10
if id_name=="MNIST" and id_model=="LeNet":
    layer_inx = 0
    OOD_DS_LIST       = OOD_DS_LIST_MNIST
    #**********************************************************
elif id_name=="CIFAR10":
    OOD_DS_LIST      = OOD_DS_LIST_CIFAR10
    if id_model=="VGG":
        layer_inx = 6
    elif id_model=="ResNet":
        layer_inx = 20
    #**********************************************************
elif id_name=="SVHN":
    OOD_DS_LIST      = OOD_DS_LIST_SVHN
    if id_model=="VGG":
        layer_inx = 6
    elif id_model=="ResNet":
        layer_inx = 24

In [ ]:
# Vriance模块(OODL大类别 --> 删除同样的feature maps)
test = np.asarray(sc_id)
for i, ood_ds_name in enumerate(OOD_DS_LIST):
    (_,_),(ood_eva_data,_) = load_dataset(ood_ds_name)
    das = preprocess_images(id_name, ood_eva_data, id_model, verbose=True)
    
    # scaler.fit(das.flatten().reshape(das.shape[0],das.shape[1]*das.shape[2]*das.shape[3]))
    # das_0 = scaler.transform(das.flatten().reshape(das.shape[0],das.shape[1]*das.shape[2]*das.shape[3]))
    # das = das_0.reshape(das.shape[0],das.shape[1],das.shape[2],das.shape[3])
    
    sc_ood = []
    for i in range(len(das)):
        xx = ae.predict(das[i].reshape(1,28,28,1))
        score = mark(das[i],xx)
        sc_ood.append(-score)
    
    ood = np.asarray(sc_ood) 
    scores, mixed_labels = combine_inliners_outliers(test, ood) 

    fpr, tpr = nums(scores, mixed_labels)
    roc_auc = auc(fpr, tpr)
    lens = id_eva_data_processed.shape[0]
    FP,TN,TP,FN = ErrorRateAt95Recall1(lens, scores, mixed_labels)
    ROC = roc_auc_score(mixed_labels, scores, average='micro', sample_weight=None)
#     print(get_summary_statistics(scores, mixed_labels))
    print('The results of %s:\n' %(ood_ds_name))
#     print('fpr_at_95_tpr:', float(FP) / float(FP + TN+ 1e-7) *100)
    print('tnr_at_95_tpr:', float(TN) / float(FP + TN+ 1e-7) *100)
    print('detection_acc:',(float(TP) / float(TP + FN + 1e-7)+ float(TN) / float(FP + TN + 1e-7))/2*100)
#     print('detection_errror:',(1.0- float(TP) / float(TP + FN+ 1e-7)+ float(FP) / float(FP + TN+ 1e-7))/2*100)
    print("AUROC: ",ROC*100)
    print("\n")

In [ ]:
ood_ds_name = 'foolingimages'

das = np.load('adv_datasets/%s_%s_fooling_images.npy'%(id_name,id_model))

# scaler.fit(das.flatten().reshape(das.shape[0],das.shape[1]*das.shape[2]*das.shape[3]))
# das_0 = scaler.transform(das.flatten().reshape(das.shape[0],das.shape[1]*das.shape[2]*das.shape[3]))
# das = das_0.reshape(das.shape[0],das.shape[1],das.shape[2],das.shape[3])

sc_ood = []
for i in range(len(das)):
        xx = ae.predict(das[i].reshape(1,28,28,1))
        score = mark(das[i],xx)
        sc_ood.append(-score)

ood = np.asarray(sc_ood) 
scores, mixed_labels = combine_inliners_outliers(test, ood) 

# print('scores is:', (scores[10000:20000]))
fpr, tpr = nums(scores, mixed_labels)
roc_auc = auc(fpr, tpr)
lens = id_eva_data_processed.shape[0]
FP,TN,TP,FN = ErrorRateAt95Recall1(lens,scores, mixed_labels)
ROC = roc_auc_score(mixed_labels, scores, average='micro', sample_weight=None)
print('The results of %s:\n' %(ood_ds_name))
# print(get_summary_statistics(scores, mixed_labels))
# print('fpr_at_95_tpr:', float(FP) / float(FP + TN+ 1e-7) *100)
print('tnr_at_95_tpr:', float(TN) / float(FP + TN+ 1e-7) *100)
print('detection_acc:',(float(TP) / float(TP + FN + 1e-7)+ float(TN) / float(FP + TN + 1e-7))/2*100)
# print('detection_errror:',(1.0- float(TP) / float(TP + FN+ 1e-7)+ float(FP) / float(FP + TN+ 1e-7))/2*100)
print("AUROC: ",ROC*100)
print("\n")